In data collecting stage, we will parse 2 channels via Telegram API.
1. t.me/bbbreaking
2. t.me/rbc_news

After that we will preprocess data to get clear text messages with that we will work in next stages of our resarch.

NB! To reproduce this code you need Telegram API developer account. We recommend you not to run code frames without it. 

# Parsing

In [1]:
from telethon.sync import TelegramClient
from telethon.tl.functions.messages import GetHistoryRequest
from telethon.tl.functions.contacts import ResolveUsernameRequest
import emoji
import getpass
import pandas as pd
from alphabet_detector import AlphabetDetector
ad = AlphabetDetector()

In [31]:
bbbreaking_messages_list = []

async with TelegramClient(session = None, api_id=getpass.getpass('API id '), 
                          api_hash=getpass.getpass('API hash ')) as client:
    limit_num = 80000
    msgs = await client.get_messages('bbbreaking', limit=limit_num)
    for i in range(limit_num):
        bbbreaking_messages_list.append(msgs[i].message)

API id ········
API hash ········
Please enter your phone (or bot token): 79160304255
Please enter the code you received: 11567
Signed in successfully as Ruslan


In [33]:
rbcnews_messages_list = []

async with TelegramClient(session = None, api_id=getpass.getpass('API id '), 
                          api_hash=getpass.getpass('API hash ')) as client:
    limit_num = 70000
    msgs = await client.get_messages('rbc_news', limit=limit_num)
    for i in range(limit_num):
        rbcnews_messages_list.append(msgs[i].message)

API id ········
API hash ········
Please enter your phone (or bot token): 79160304255
Please enter the code you received: 99927
Signed in successfully as Ruslan


# Preprocessing

In [34]:
def text_normalizer(text):
    new_text = []
    for message in text:
        if message != None and len(message) > 0: # убираем пустые сообщения
            for symbol in ('\n', '\u200b', '\xa0', '/', '•', '.', ',', ':', ';', 
                           '-', '–', '—', '?', '!', '%',
                           '"', '«', '»', '(', ')', '[', ']', '$'):
                message = message.replace(symbol, ' ') # убираем знаки пунктуации и разрывы строк
            message = message.strip()
            message = ''.join(word for word in message if not emoji.is_emoji(word)) # убираем эмоджи
            message = ''.join(word for word in message if not word.isdigit()) # убираем цифры
            message = ''.join(word for word in message if ad.is_cyrillic(word)) # убираем латинские буквы
            new_text.append(message)
        
    return new_text

In [38]:
bbbreaking = pd.DataFrame(text_normalizer(bbbreaking_messages_list), columns=['message'])
bbbreaking.head()

,message
0,Суд в США предписал Трампу выплатить компенсац...
1,️Суд в США признал Трампа виновным по делу о п...
2,️Взрывы прогремели в Сумской и Черниговской об...
3,Агентство Франс Пресс сообщило о гибели своего...
4,Вторая мировая война завершилась победой США н...


In [40]:
rbc_news = pd.DataFrame(text_normalizer(rbcnews_messages_list), columns=['message'])
rbc_news.head()

,message
0,️Экс глава таможни перейдет в Совет Федерации ...
1,Суд признал Трампа виновным в изнасиловании пи...
2,В Москве дают салют в честь й годовщины побед...
3,ЕС могут включить в новый пакет санкций компан...
4,В Белгородской области под обстрел снова попал...
